In [ ]:
## The following is for google colab
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
## The following is for google colab
%cd /content/drive/My\ Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
BIG_TITLE="SVM movement"

In [ ]:
### TIME WINDOWS
w1=512
w2=768
W1=w1
W2=w2

In [ ]:
### LIBRARIES

import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy
import datetime
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
import tensorflow as tf
import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import time
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas
import os
import csv

# Remove some unwanted warnings
import logging
logging.getLogger('tensorflow').disabled = True 
from scipy.signal import butter, lfilter
import pywt
import pandas as pd
import numpy as np
import random
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy
import scipy
import scipy.signal
from scipy import fft
### LIBRARIES

import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy
import datetime
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
import tensorflow as tf
import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import time
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy
import pandas
import os

import os
import sys

import pandas as pd
import pandas
import numpy as np

import pywt
import scipy.io as spio
from scipy.stats import entropy
from collections import Counter

from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import timeit

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import LSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
import tensorflow as tf
import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from numpy import hstack, vstack, dstack
import numpy as np


In [ ]:
def load_signal(k):  #where k is the file name
    data = sio.loadmat(str(k))  
    ###deleting EOGs
    signal_raw= numpy.delete(data["signal"], obj= [63,62,61,60,59,58], axis=0)
    ###deleting 0s:
    signal_raw_trans= numpy.transpose(signal_raw)
    ind=[]
    for cx, c in enumerate(signal_raw_trans):
        if len(c) -  numpy.count_nonzero(c) <=5:
            continue
        else:
            ind.append(cx)
    g= numpy.delete(signal_raw_trans, obj= ind, axis=0)
    signal_raw_trans_clean=numpy.transpose(g)
    
    ###clean memory
    del data
    del signal_raw
    del signal_raw_trans
    del g
    return(signal_raw_trans_clean)

In [ ]:
### cleaned code names
def cleaned_code_names(k):# k=file name
    data = sio.loadmat(str(k)) 
    events= data["header"]["event_codes"]
    EVENTS=[]
    for i in events:
        for e in i:
            for j in e:
                for h in j:
                    if len(str(h))>2:
                        EVENTS.append(h)
    EVENTS=numpy.array(EVENTS)
    
    del data
    del events
    
    return EVENTS

In [ ]:
### cleaned event code
def event_Code(k): # k=file name
    data = sio.loadmat(str(k)) 
    labels= str(data["events"]["codes"]).split()
    
    l=[]
    for g in labels:
        for i in g:
            if i.isnumeric():
                l.append(i)
        else:
            l.append(",")
    #cleaning labels 
    l2=[]
    g=""
    for j in l:
        if j.isnumeric():
            g+=j
        elif len(g)<1:
            continue
        else:
            l2.append(int(g))
            g=""
    del data
    del labels
    del l
    del g
    
    return l2

### cleaned event position
def event_Position(k):# k=file name
    data = sio.loadmat(str(k)) 
    indices_signal_matrix= str(data["events"]["positions"]).split()
    #cleaning indices_signal_matrix 
    l3=[]
    for g in indices_signal_matrix:
        for i in g:
            if i.isnumeric():
                l3.append(i)
        else:
            l3.append(",")
    l4=[]
    g=""
    for j in l3:
        if j.isnumeric():
            g+=j
        elif len(g)<1:
            continue
        else:
            l4.append(int(g))
            g=""
    l4=l4[:-1]
    del data
    del indices_signal_matrix
    del l3
    del g
    
    return l4

In [ ]:
#indexes of each events within signals
def indexes_event_signals(k):
    data = sio.loadmat(str(k)) 
    EVENTS = cleaned_code_names(k)
    event_codes= event_Code(k)
    dico_indexes={}
    for event in EVENTS:
        indexes_=[]
        for ig, g in enumerate(event_codes):
            if g == event:
                indexes_.append(ig)
        dico_indexes[event]= indexes_
    del data
    del EVENTS
    del event_codes
    del indexes_
    return dico_indexes

In [ ]:
def start_indexes(k, event):
    indexes=indexes_event_signals(k)
    event_positions= event_Position(k)
    event_indexes=[]
    for h in indexes[event]:
        event_indexes.append(event_positions[h])
    del indexes
    del event_positions
    return event_indexes

In [ ]:
def EVENT_SIGNALS(k, EVENT, W1, W2):
    signal= load_signal(str(k)) 
    event_indexs= start_indexes(str(k), EVENT)
    movement_signals=[]
    for i in event_indexs:#
        for channel in signal:
            i= int(i)
            if i-W1>0:
                movement_signals.append(numpy.array(channel[i-W1:i+W2]))
            else:
                continue
    del signal
    del event_indexs
    
    return movement_signals


In [ ]:
def create_tensor(data,W1:int, W2,nb_channels:int):
    window_size= W1+W2
    positions_split=[]
    for j in range(0,len(data),nb_channels):
        positions_split.append(j)
    positions_split.append(len(data))
    all_events=[]
    for i in range(0,len(positions_split)-1):
        one_event=[]
        for h in range(positions_split[i],positions_split[i+1]):
            data_shaped=data[h]
            if (len(one_event)==0.0):
                one_event=data_shaped
            else:
                one_event=numpy.vstack((one_event, data_shaped))
        if (len(all_events)==0.0):
                all_events=one_event.reshape(1,nb_channels,window_size)
        else:
                all_events=numpy.vstack((all_events, one_event.reshape(1,nb_channels,window_size)))
    return(all_events)

In [ ]:
def random_shuffle(final_TENSOR, labels):
    random_indexes_tens= random.sample(range(0, final_TENSOR.shape[0]), final_TENSOR.shape[0])
    event_tensors= [t for t in range(final_TENSOR.shape[0])]#indexes of tensor
    SHUFFLED_TENSOR=[] 
    LABELS_SHUFFLED=[]
    
    for i in event_tensors:
        target_position=random_indexes_tens[i]
        SHUFFLED_TENSOR.append(final_TENSOR[target_position,:,:])
        LABELS_SHUFFLED.append(labels[target_position])
    SHUFFLED_TENSOR=numpy.array(SHUFFLED_TENSOR)
    return SHUFFLED_TENSOR, LABELS_SHUFFLED

In [ ]:

def add_data(W1:int, W2:int,k:str,event_label:str,EVENT,nb_channels:int):
    #window_size=W1+W2
    partial_TENSOR=[]
    partial_LABEL=[]
    time00 = datetime.datetime.now()
    #for i in k:
    time1 = datetime.datetime.now()
    file='G'+k+'.mat'
    print('processing '+file)
    data=EVENT_SIGNALS(k=file, EVENT=EVENT, W1=W1, W2=W2 )
    data_transformed=create_tensor(data=data,W1=W1, W2=W2,nb_channels=nb_channels)
    label= event_label*data_transformed.shape[0]
    
    if (len(partial_TENSOR)==0.0):
        partial_TENSOR=data_transformed
        partial_LABEL=label
    else:
        partial_TENSOR=numpy.vstack((partial_TENSOR, data_transformed))
        partial_LABEL=partial_LABEL+label
    print('New shape Tensor ', partial_TENSOR.shape)
    print('New shape label ', len(partial_LABEL))
    time2 = datetime.datetime.now()
    elapsedTime = time2 - time1
    print('Minutes it took',elapsedTime)
        
    time01 = datetime.datetime.now()
    elapsedTime0 = time01 - time00
    print('Total time',elapsedTime0)

    return(partial_TENSOR, partial_LABEL) 

In [ ]:
data = sio.loadmat(str("G01.mat"))
EEG_signals= load_signal("G01.mat")
header_channels_labels= data["header"]["channels_labels"]#living out EOGs
header_channels_labels_filtered=[]
for i in header_channels_labels:
    for j in i:
        for s in j:
            for ij in s:
                for ix in ij:
                    header_channels_labels_filtered.append(str(ix).strip())
header_channels_labels_filtered=header_channels_labels_filtered[:-6]

In [ ]:
#band pass filter between 0.5 and 60 hz

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
ch=58
nb_ch=58
fs = 256.0
lowcut = 0.5
highcut = 60.0
a = 0.02
f0 = 256.0
T = 1.0 / 256.0
w = pywt.Wavelet('db4')

def get_Filter_FeatExtract(file,W1,W2, nb_ch):
    P1_reach= EVENT_SIGNALS(file, EVENT= 503587, W1=W1, W2=W2)
    P1_reach2=EVENT_SIGNALS(file, EVENT= 503588, W1=W1, W2=W2)
    P1_grasp= EVENT_SIGNALS(file, EVENT= 501794, W1=W1, W2=W2)
    P1_grasp2=EVENT_SIGNALS(file, EVENT= 501795, W1=W1, W2=W2)
    
    P1_reach= create_tensor(P1_reach,W1=W1, W2=W2,nb_channels=nb_ch)
    P1_reach2=create_tensor(P1_reach2,W1=W1, W2=W2,nb_channels=nb_ch)
    P1_grasp=create_tensor(P1_grasp,W1=W1, W2=W2,nb_channels=nb_ch)
    P1_grasp2=create_tensor(P1_grasp2,W1=W1, W2=W2,nb_channels=nb_ch)
         

    for event_i in range(0,P1_reach.shape[0]):  
      y_r = butter_bandpass_filter(P1_reach[event_i], lowcut, highcut, fs, order=6)
      #array_r = Implement_Notch_Filter(T, 1, 60, 1, 2, 'butter', y_r)
      maxlev = pywt.dwt_max_level(len(y_r), w.dec_len)
      # maxlev = 2 # Override if desired
      #print("maximum level is " + str(maxlev)) #max level is 3
      threshold = 0.04 #
      data= y_r
      coeffs = pywt.wavedec(data, 'db4', level=maxlev)# 
      datarec = pywt.waverec(coeffs, 'db4')
      try:
        partial_reachF=numpy.vstack((partial_reachF,datarec.reshape(1, datarec.shape[0], datarec.shape[1])))
      except NameError:
        partial_reachF=datarec.reshape(1, datarec.shape[0], datarec.shape[1])

    del y_r
    del P1_reach

    for event_i in range(0,P1_reach2.shape[0]): 
      y_r = butter_bandpass_filter(P1_reach2[event_i], lowcut, highcut, fs, order=6)
      #array_r = Implement_Notch_Filter(T, 1, 60, 1, 2, 'butter', y_r)
      maxlev = pywt.dwt_max_level(len(y_r), w.dec_len)
      # maxlev = 2 # Override if desired
      threshold = 0.04 #
      data= y_r
      coeffs = pywt.wavedec(data, 'db4', level=maxlev)# 
      datarec = pywt.waverec(coeffs, 'db4')
      try:
        partial_reachFF=numpy.vstack((partial_reachFF,datarec.reshape(1, datarec.shape[0], datarec.shape[1])))
      except NameError:
        partial_reachFF=datarec.reshape(1, datarec.shape[0], datarec.shape[1])

    del y_r
    del P1_reach2

    for event_i in range(0,P1_grasp.shape[0]):  
      y_r = butter_bandpass_filter(P1_grasp[event_i], lowcut, highcut, fs, order=6)
      maxlev = pywt.dwt_max_level(len(y_r), w.dec_len)
      # maxlev = 2 # Override if desired
      threshold = 0.04 #
      data= y_r
      coeffs = pywt.wavedec(data, 'db4', level=maxlev)# 
      datarec = pywt.waverec(coeffs, 'db4')
      try:
        partial_graspF=numpy.vstack((partial_graspF,datarec.reshape(1, datarec.shape[0], datarec.shape[1])))
      except NameError:
        partial_graspF=datarec.reshape(1, datarec.shape[0], datarec.shape[1])

    del y_r
    del P1_grasp

    for event_i in range(0,P1_grasp2.shape[0]): 
      y_r = butter_bandpass_filter(P1_grasp2[event_i], lowcut, highcut, fs, order=6)
      #array_r = Implement_Notch_Filter(T, 1, 60, 1, 2, 'butter', y_r)
      maxlev = pywt.dwt_max_level(len(y_r), w.dec_len)
      # maxlev = 2 # Override if desired
      #print("maximum level is " + str(maxlev)) #max level is 3
      threshold = 0.04 #
      data= y_r
      coeffs = pywt.wavedec(data, 'db4', level=maxlev)# 
      datarec = pywt.waverec(coeffs, 'db4')
      try:
        partial_graspFF=numpy.vstack((partial_graspFF,datarec.reshape(1, datarec.shape[0], datarec.shape[1])))
      except NameError:
        partial_graspFF=datarec.reshape(1, datarec.shape[0], datarec.shape[1])

    del y_r
    del P1_grasp2
    

    partial_reach= numpy.vstack([partial_reachF, partial_reachFF])
    labels_reach= ["reach"]*partial_reach.shape[0]
    partial_grasp=numpy.vstack([partial_graspF, partial_graspFF])
    labels_grasp=["grasp"]*partial_grasp.shape[0]
    return partial_reach, labels_reach, partial_grasp, labels_grasp


In [ ]:
# Function to compute the classification using SVM
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
def compute_SVC(train_f,train_l):
	C=1.0
	cache_size=200
	class_weight=None
	coef0=0.0
	decision_function_shape=None
	degree=3
	gamma='auto'
	kernel='rbf'
	max_iter=-1
	probability=False
	random_state=None
	shrinking=True
	tol=0.001
	verbose=False
	c = svm.SVC(kernel='rbf')
	c.fit(train_f,train_l)
	return c;
	
# Function to calculate the accuracy
def svm_results(test_f,test_l,c):
  pred = c.predict(test_f)
	#print(pred)
  Accuracy_Score = accuracy_score(test_l, pred)
  Precision_Score = precision_score(test_l, pred,  average="macro")
  Recall_Score = recall_score(test_l, pred,  average="macro")
  F1_Score = f1_score(test_l, pred,  average="macro")
  CM = confusion_matrix(test_l, pred)
  return Accuracy_Score, Precision_Score, Recall_Score, F1_Score, CM, pred

In [ ]:
import functools
def combine_dims(a, i=0, n=1):
  """
  Combines dimensions of numpy array `a`, 
  starting at index `i`,
  and combining `n` dimensions
  """
  s = list(a.shape)
  combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
  return np.reshape(a, s[:i] + [combined] + s[i+n+1:])

#imgs = combine_dims(FINAL_TENSOR, 1)#combines dimension 1 and 2

In [ ]:
###
nb_ch=58
training_time=[]
test_accuracies=[]
#test_f1score=[]
files= ['G01.mat','G02.mat','G03.mat','G04.mat','G05.mat','G06.mat','G07.mat','G08.mat','G09.mat','G10.mat','G11.mat','G12.mat','G13.mat','G14.mat','G15.mat']

# .....................
for g in range(len(files)):
  train_files= files[:g]+ files[g+1:]
  test_file=[files[g]]

  for file in train_files:
    partial_reach, labels_reach, partial_grasp, labels_grasp= get_Filter_FeatExtract(file,W1,W2, nb_ch)
    beside= vstack((partial_reach,partial_grasp))
    beside2=labels_reach+labels_grasp
    try:
      FINAL_TENSOR= vstack((FINAL_TENSOR, beside))
      FINAL_LABELS= FINAL_LABELS+beside2
    except NameError:
      FINAL_TENSOR=beside
      FINAL_LABELS=beside2

  X_train, X_val, y_train, y_val = train_test_split(FINAL_TENSOR, FINAL_LABELS, test_size =0.3, random_state=0)
  X_train=numpy.vstack((X_train,X_val))
  print("x train shape", X_train.shape)
  y_train=y_train+y_val
  print("y train length", len(y_train))

  del FINAL_TENSOR
  del FINAL_LABELS
  del beside
  del beside2

  X_train = combine_dims(X_train, 1)
  onehot = LabelBinarizer()
  y_train   = onehot.fit_transform(y_train)

  from sklearn import svm

  start = time.time()

  model_svc = compute_SVC(X_train, y_train.ravel())

  stop = time.time()  
  training_time.append(stop - start)

# ### END TRAIN ##
  del X_train
  del X_val
  del y_train
  del y_val

  for file in test_file:
    partial_reach, labels_reach, partial_grasp, labels_grasp= get_Filter_FeatExtract(file,W1,W2, nb_ch)
    besidE= vstack((partial_reach,partial_grasp))
    besidE2=labels_reach+labels_grasp
    try:
      TEST_TENSOR= vstack((TEST_TENSOR, besidE))
      TEST_LABELS= TEST_LABELS+besidE2
    except NameError:
      TEST_TENSOR=besidE
      TEST_LABELS=besidE2

  X_train, X_val, y_train, y_val = train_test_split(TEST_TENSOR, TEST_LABELS, test_size =0.3, random_state=0)
  X_train=numpy.vstack((X_train,X_val))
  print("x test shape before flattening", X_train.shape)
  y_train=y_train+y_val
  print("y test length", len(y_train))


  X_train = combine_dims(X_train, 1)#combines dimension 1 and 2
  print("x test shape after flattening", len(X_train))#.shape)
  onehot = LabelBinarizer()
  y_train   = onehot.fit_transform(y_train)

  from sklearn import svm
	
  TAccuracy_Score, TPrecision_Score, TRecall_Score, TF1_Score, TCM, Tpred = svm_results(X_train,y_train,model_svc);

  test_accuracies.append(TAccuracy_Score)
  #test_f1score.append(TF1_Score)
  
  TITLE= "Test results & training time "+BIG_TITLE+ "K-fold number "+ str(g)
  with open(str(TITLE)+"K-fold number"+ str(g)+".csv", 'w', newline='', ) as file:
      writer = csv.writer(file)
      writer.writerow(["Test acc:", TAccuracy_Score])
      writer.writerow(["sensitivity/recall:", TRecall_Score])
      writer.writerow(["CM:", TCM])
      writer.writerow(["precision",TPrecision_Score])#
      #writer.writerow(["F1-score",TF1_Score])
      #writer.writerow(["Training time:", training_time])


  del TEST_TENSOR
  del TEST_LABELS
  del besidE      
  del besidE2

x train shape (4338, 58, 1280)
y train length 4338
x test shape before flattening (316, 58, 1280)
y test length 316
x test shape after flattening 316
x train shape (4354, 58, 1280)
y train length 4354
x test shape before flattening (300, 58, 1280)
y test length 300
x test shape after flattening 300
x train shape (4338, 58, 1280)
y train length 4338
x test shape before flattening (316, 58, 1280)
y test length 316
x test shape after flattening 316
x train shape (4352, 58, 1280)
y train length 4352
x test shape before flattening (302, 58, 1280)
y test length 302
x test shape after flattening 302
x train shape (4334, 58, 1280)
y train length 4334
x test shape before flattening (320, 58, 1280)
y test length 320
x test shape after flattening 320
x train shape (4336, 58, 1280)
y train length 4336
x test shape before flattening (318, 58, 1280)
y test length 318
x test shape after flattening 318
x train shape (4346, 58, 1280)
y train length 4346
x test shape before flattening (308, 58, 1280)
y 

In [ ]:
import math

def variance(data, ddof=0):
  n = len(data)
  mean = sum(data) / n
  return sum((x - mean) ** 2 for x in data) / (n - ddof)

def stdev(data):
  var = variance(data)
  std_dev = math.sqrt(var)
  return std_dev

def Average(lst):
    return sum(lst) / len(lst)

SE_testacc= stdev(test_accuracies)
#SE_testf1= stdev(test_f1score)
Mean_accuracy= Average(test_accuracies)
#Mean_f1score= Average(test_f1score)

with open(str(BIG_TITLE+"SE")+".csv", 'w', newline='', ) as file:
      writer = csv.writer(file)
      writer.writerow(["SE_test accuracy", SE_testacc])
      writer.writerow(["Mean_accuracy", Mean_accuracy])
      #writer.writerow(["SE_test f1score", SE_testf1])
      #writer.writerow(["Mean_f1score", Mean_f1score])

In [ ]:
print("SE acc", SE_testacc*100)
#print("SE F1", SE_testf1*100)

In [ ]:
print("Mean acc", Mean_accuracy*100)
#print("Mean F1 score", Mean_f1score*100)

In [ ]:
test_accuracies

[0.629746835443038,
 0.73,
 0.6234177215189873,
 0.6920529801324503,
 0.590625,
 0.5628930817610063,
 0.6915584415584416,
 0.621875,
 0.6858974358974359,
 0.7597402597402597,
 0.710691823899371,
 0.6037735849056604,
 0.7115384615384616,
 0.7553956834532374,
 0.6590909090909091]

In [ ]:
#test_f1score